# "Ambiguity sink" in remapping pipelines
## A complementary phenomena to what is better described as "ambiguity source"
### April 17th, 2025

In pipelines with a step that involves remapping to consensus, an initial reference must be supplied. Genetic dissimilarity/divergence in this reference compared to the sample to be assembled can result in reads not being mapped. This can masquerade as dropout, or regions that failed to sequence, on the wet lab side. However, it's well-known that the chosen reference can bias an analysis. Choosing a more similar reference may result in more reads mapping to a homologous but more genetically similar region, showing that some dropped out regions may in fact be computational artifacts.

In the current HPAI panzootic, there is an astonishing amount of circulating diversity even in geographically limited areas. This begs the question of how to choose a reference to assemble multiple samples from such a region. The idea behind remapping pipelines is that an initial reference can have a per-sample consensus called and remapped to. This new consensus sequence can ideally serve as exactly genetically similar to what is in the sample. However, this strategy is not without challenges. This notebook describes a complementary phenomena to yesterday's notebook. There, what could better be described as "ambiguity source" was explored: in regions of remapped consensus calls with stretches of ambiguous bases, without a fill-in step, these ambiguous regions can actually <i>slowly expand</i> upon each remapping step. Here we describe a complementary phenomena: with fill-in, these regions <i>slowly contract</i> with each remapping. This can be described as ambiguity sinking.

This motivates the use of tools like [VAPOR](https://academic.oup.com/bioinformatics/article/36/6/1681/5613803). VAPOR takes a FASTQ dataset for an individual sample and a FASTA of candidate references as input, and outputs a single sequence to be used as a genetically similar reference for that sample. This is in contrast to using a single reference for an initial consensus call for every sample in an entire project. This notebooks shows that this dramatically accelerates filling in when compared with the strategy of multiple remappings, producing better consensus sequences with far less computation. However, even this strategy alone has it doubts. We raise the question: is a combination of VAPOR and multiple remappings to achieve maximum ambiguity sink the ideal strategy in the current HPAI panzootic?

## Exploring VAPOR

We start from the paired and unpaired forward and reverse reads from the `trimmomatic` step of the pipeline for the `ms_w2` `pa` replicate 1  from yesterday's analysis. They are concatenated and fed to VAPOR. We search for a suitable reference using PA sequences from [our NA build](https://github.com/moncla-lab/nextstrain-hpai-north-america/).

In [1]:
cat *.fastq > concat.fastq
vapor.py -fq concat.fastq -fa aligned_pa.fasta

Loading database sequences
Got 4136 unique sequences
Getting database kmers
Got 81290 database kmers
Filtering reads
50009 of 818099 reads survived
Building wDBG
Got 48100 wdbg kmers
Culling kmers with coverage under 5 
3084 kmers remaining
Classifying
0.9995433789954338	4348080.0	2190	1985.4246575342465	50009	>A/eagle/Virginia/W221222/2022


We then pluck this out to be used as a reference and run the core steps of the pipeline at present.

In [2]:
seqkit grep -p 'A/eagle/Virginia/W221222/2022' aligned_pa.fasta > reference.fasta

bowtie2-build reference.fasta ref 
bowtie2 --local --very-sensitive-local -x ref -1 forward_paired.fastq -2 reverse_paired.fastq -U forward_unpaired.fastq,reverse_unpaired.fastq -S mapped.sam

samtools view -S -b mapped.sam > mapped.bam
samtools sort mapped.bam -o sorted.bam
samtools index sorted.bam
samtools mpileup -A -B -Q 0 -d 100000 -f reference.fasta sorted.bam > pileup.txt

cat pileup.txt | ivar consensus -p ivar -m 50 -q 30 -t .0

Settings:
  Output files: "ref.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fasta
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 547
Using parameters --bmax 411 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 411 --dcv 1024
Constructing suffix-array element generator


Multiple remappings were implemented as follows, making adjusts to the rules to call sample consensus and call consensus in batches:

    +NUMBER_OF_REMAPPINGS = 3
    +
     wildcard_constraints:
       segment="[^/]+",
       sample="[^/]+",
    @@ -184,10 +186,10 @@ rule trimmomatic:
     def situate_reference_input(wildcards):
         if wildcards.mapping_stage == 'initial':
             return 'data/reference/sequences.fasta'
    -    elif wildcards.mapping_stage == 'remapping':
    +    elif wildcards.mapping_stage == 'remapping-1':
             return f'data/{wildcards.sample}/replicate-{wildcards.replicate}/initial/filler.fasta'
    -    else:
    -        return f'data/{wildcards.sample}/replicate-{wildcards.replicate}/remapping/filler.fasta'
    +    mapping_stage_int = int(wildcards.mapping_stage.split('-')[1]) - 1
    +    return f'data/{wildcards.sample}/replicate-{wildcards.replicate}/remapping-{mapping_stage_int}/filler.fasta'
     
     
     rule situate_reference:
    @@ -441,7 +443,7 @@ rule fill_consensus:
     
     def call_sample_consensus_input(wildcards):
         return expand(
    -        'data/{{sample}}/replicate-{replicate}/reremapping/consensus.fasta',
    +        f'data/{{sample}}/replicate-{replicate}/remapping-{NUMBER_OF_REMAPPINGS}/consensus.fasta',
             replicate=metadata_dictionary[wildcards.sample].keys()
         )
     
    @@ -582,7 +584,7 @@ def full_consensus_summary_input(wildcards):
         for replicate in replicates.keys():
             for segment in SEGMENTS:
                 consensus_filepaths.append(
    -                f'data/{wildcards.sample}/replicate-{replicate}/reremapping/segments/{segment}/consensus-report.tsv'
    +                f'data/{wildcards.sample}/replicate-{replicate}/remapping-{NUMBER_OF_REMAPPINGS}/segments/{segment}/consensus-report.tsv'
                 )
         return consensus_filepaths

Below we display 

- three remappings from a sequence that is diverged from our initial reference
- an initial consensus called with VAPOR's reference without any remapping at all

<img src="images/003-three-remappings-vs-vapor.png" width="1140">

This shows that the reference selected by VAPOR manages to call an additional ~400 bp than the third remapping, and that the remapping strategy as implemented at present only calls about an additional dozen bases per remapping as implemented at present.

It also raises the following questions
- Would remapping help resolve the ~60 bp region near 1105-1175?
- Is there <b>still</b> signal hidden amongst this noise?
- How many remappings would be necessary?